[View in Colaboratory](https://colab.research.google.com/github/Joovvhan/ColabTest/blob/master/Transfer_Learning_Model_Comparison.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.io.wavfile as wf

In [3]:
os.listdir('gdrive/My Drive/Colab')

['Data', 'Model']

In [0]:
dataPath = 'gdrive/My Drive/Colab/Data'

"""
folders = os.listdir(dataPath)

for folder in folders:
  files = os.listdir(dataPath + '/' + folder)
  print(len(files))
  print(files[-1])
"""

folderF1 = dataPath + '/' + 'A3F1P3'  
folderF5 = dataPath + '/' + 'A3F5P3'
filesF1 = os.listdir(folderF1)
filesF5 = os.listdir(folderF5)

folderA5 = dataPath + '/' + 'A5F3P3'
filesA5 = os.listdir(folderA5)

In [5]:
filesF1[0]

'A3.490879_F0.139398_P2.076554_H360.426598.wav'

In [0]:
fs, dataInt16 = wf.read(folderF1 + '/' + filesF1[0])
dataFloat = dataInt16 / (2 ** 15)

fs, dataInt16 = wf.read(folderF5 + '/' + filesF5[0])
dataFloat = dataInt16 / (2 ** 15)

In [0]:
nsc = 1470
nov = nsc/2
nff = nsc 

In [0]:
Pxx, freqs, bins, _ = plt.specgram(dataFloat, NFFT=nff, Fs=fs, noverlap=nov, \
                                   window=np.hamming(nsc), cmap='viridis')
plt.close()
img = Pxx[0:224, :]

plt.imshow(10*np.log10(img), cmap='viridis')
plt.close()

In [0]:
import time

In [0]:
from tqdm import tqdm, trange

In [11]:
imgSize = 224
imgsF1 = np.zeros([len(filesF1), imgSize, imgSize])

for i in trange(len(filesF1)):
    fs, dataInt16 = wf.read(folderF1 + '/' + filesF1[i])
    dataFloat = dataInt16 / (2 ** 15)
    Pxx, _, _, _ = plt.specgram(dataFloat, NFFT=nff, Fs=fs, noverlap=nov, \
                                       window=np.hamming(nsc), cmap='viridis')
    plt.close()
    imgsF1[i, :, :] = 10 * np.log10(Pxx[0:224, :])

100%|██████████| 1000/1000 [01:13<00:00, 14.42it/s]


In [12]:
imgsF5 = np.zeros([len(filesF5), imgSize, imgSize])

for i in trange(len(filesF5)):
    fs, dataInt16 = wf.read(folderF5 + '/' + filesF5[i])
    dataFloat = dataInt16 / (2 ** 15)
    Pxx, _, _, _ = plt.specgram(dataFloat, NFFT=nff, Fs=fs, noverlap=nov, \
                                       window=np.hamming(nsc), cmap='viridis')
    plt.close()
    imgsF5[i, :, :] = 10 * np.log10(Pxx[0:224, :])

100%|██████████| 1000/1000 [01:14<00:00, 14.16it/s]


In [0]:
trainIdxF1 = np.random.choice(999, int(1000 * 0.8), replace=False)
testIdxF1 = list(set(range(0, 1000)) - set(trainIdxF1))

trainImgsF1 = imgsF1[trainIdxF1, :, :]
testImgsF1 = imgsF1[testIdxF1, :, :]

trainIdxF5 = np.random.choice(999, int(1000 * 0.8), replace=False)
testIdxF5 = list(set(range(0, 1000)) - set(trainIdxF5))

trainImgsF5 = imgsF5[trainIdxF5, :, :]
testImgsF5 = imgsF5[testIdxF5, :, :]

In [0]:
trainImgs = np.vstack([trainImgsF1, trainImgsF5])
testImgs = np.vstack([testImgsF1, testImgsF5])

In [0]:
trainMean = np.mean(trainImgs)
trainStd = np.std(trainImgs)

In [0]:
trainImgsNorm = (trainImgs - trainMean) / trainStd
testImgsNorm = (testImgs - trainMean) / trainStd

trainImgsNorm = trainImgsNorm.reshape(list(trainImgsNorm.shape) + [1])
testImgsNorm = testImgsNorm.reshape(list(testImgsNorm.shape) + [1])

In [0]:
X_train = np.stack([trainImgsNorm[:, :, :, 0], trainImgsNorm[:, :, :, 0], trainImgsNorm[:, :, :, 0]], axis = -1)
X_test = np.stack([testImgsNorm[:, :, :, 0], testImgsNorm[:, :, :, 0], testImgsNorm[:, :, :, 0]], axis = -1)

In [0]:
trainLabelF1 = np.stack((np.ones(800), np.zeros(800)), axis = -1)
testLabelF1 = np.stack((np.ones(200), np.zeros(200)), axis = -1)

trainLabelF5 = np.stack((np.zeros(800), np.ones(800)), axis = -1)
testLabelF5 = np.stack((np.zeros(200), np.ones(200)), axis = -1)

Y_train = np.vstack((trainLabelF1, trainLabelF5))
Y_test = np.vstack((testLabelF1, testLabelF5))

In [0]:
from keras.preprocessing import image
from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Sequential

In [0]:
from keras.applications import VGG16

input_tensor = Input(shape=(224, 224, 3))

modelVGG16WoTop = None
modelVGG16 = None

modelVGG16WoTop = VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)

modelVGG16 = Sequential()

modelVGG16.add(modelVGG16WoTop)
modelVGG16.add(Flatten())
modelVGG16.add(Dense(4096, activation='relu'))
# modelVGG16.add(Dropout(0.5))
modelVGG16.add(Dense(4096, activation='relu'))
# modelVGG16.add(Dropout(0.5))

modelVGG16.add(Dense(2, activation='softmax'))

modelVGG16.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

modelVGG16.fit(X_train, Y_train,
          batch_size=2, epochs=8, verbose=1,
          validation_data=(X_test, Y_test))

Train on 1600 samples, validate on 400 samples
Epoch 1/8
1600/1600 [==============================] - 254s 159ms/step - loss: 8.0160 - acc: 0.4994 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 2/8
 792/1600 [=============>................] - ETA: 1:59 - loss: 7.5091 - acc: 0.5316

In [19]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

modelResNet50WoTop = None
modelResNet50 = None

input_tensor = Input(shape=(224, 224, 3))
modelResNet50WoTop = ResNet50(input_tensor=input_tensor, weights='imagenet', include_top=False)

modelResNet50 = Sequential()

modelResNet50.add(modelResNet50WoTop)
modelResNet50.add(Flatten())
modelResNet50.add(Dense(2, activation='softmax'))

modelResNet50.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

modelResNet50.fit(X_train, Y_train,
          batch_size=4, epochs=8, verbose=1,
          validation_data=(X_test, Y_test))

Using TensorFlow backend.


Train on 1600 samples, validate on 400 samples
Epoch 1/2
 248/1600 [===>..........................] - ETA: 2:53 - loss: 0.4881 - acc: 0.8911

KeyboardInterrupt: ignored

In [0]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import datetime

now = datetime.datetime.now()
modelResNet50.save('gdrive/My Drive/Colab/Model/ResNet50 {}.h5'.format(now.strftime('%m-%d %H:%M:%S')))
# modelResNet50_ = load_model('gdrive/My Drive/Colab/Model/ResNet50.h5')

# Y_pred = modelResNet50_.predict(X_test)
# print(confusion_matrix(np.argmax(Y_pred, axis = 1), np.argmax(Y_test, axis = 1)))

In [0]:
from keras import backend as K
K.clear_session()

In [0]:
from keras.applications import VGG19

input_tensor = Input(shape=(224, 224, 3))

modelVGG19WoTop = None
modelVGG19 = None

modelVGG19WoTop = VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)

modelVGG19 = Sequential()

modelVGG19.add(modelVGG19WoTop)
modelVGG19.add(Flatten())
modelVGG19.add(Dense(4096, activation='relu'))
modelVGG19.add(Dropout(0.5))
modelVGG19.add(Dense(4096, activation='relu'))
modelVGG19.add(Dropout(0.5))
modelVGG19.add(Dense(2, activation='softmax'))

modelVGG19.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

modelVGG19.fit(X_train, Y_train,
          batch_size=2, epochs=8, verbose=1,
          validation_data=(X_test, Y_test))

In [0]:
# for i in range(1000, 1010):
#     plt.figure()
#     plt.imshow(X_train[i, :, :, 0]);
#     plt.plot()

